In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path = 'connect-4.csv'
classes = 3
dataset = np.genfromtxt("connect-4.csv", dtype='str', delimiter=",")
X = dataset[:,0:42]
Y = dataset[:,42]
#Y = LabelEncoder().fit_transform(Y)
#Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(67557, 42) (67557,)


In [3]:
x = np.zeros(X.shape)

for i, row in enumerate(X):
    for j, col in enumerate(row):
        if col == 'x':
            x[i,j] = 1.0
        if col == 'o':
            x[i,j] = -1.0
        if col == 'b':
            x[i,j] = 0.0

Y = LabelEncoder().fit_transform(Y)
y = to_categorical(Y)
x = MinMaxScaler().fit_transform(x)
print(x.shape,y.shape)

(67557, 42) (67557, 3)


In [4]:
from tab2img.converter import Tab2Img
model = Tab2Img()
images = model.fit_transform(x, Y)
print(images.shape)

(67557, 7, 7)


In [5]:
from PIL import Image
x = []
for i in range(len(images)):
    img = Image.fromarray(images[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x = np.array(x,dtype='float32')
print(x.shape,y.shape)

(67557, 32, 32, 3) (67557, 3)


In [6]:
X_train,X_test, Y_train,Y_test = train_test_split(x,y,test_size=0.259884)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(50000, 32, 32, 3) (50000, 3)
(17557, 32, 32, 3) (17557, 3)


In [7]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
# don't train existing weights
#for layer in pretrained_model.layers:
# layer.trainable = False
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu',name='flattenn')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(X_train,Y_train,batch_size=128, epochs=20,verbose=1,validation_data=(X_test,Y_test))

Epoch 1/20
391/391 [==============================] - 24s 61ms/step - loss: 0.6125 - accuracy: 0.7595 - val_loss: 0.7698 - val_accuracy: 0.7060
Epoch 2/20
391/391 [==============================] - 18s 46ms/step - loss: 0.4967 - accuracy: 0.8036 - val_loss: 0.4617 - val_accuracy: 0.8161
Epoch 3/20
391/391 [==============================] - 18s 47ms/step - loss: 0.4525 - accuracy: 0.8214 - val_loss: 0.4890 - val_accuracy: 0.8133
Epoch 4/20
391/391 [==============================] - 18s 47ms/step - loss: 0.4277 - accuracy: 0.8299 - val_loss: 0.5538 - val_accuracy: 0.8005
Epoch 5/20
391/391 [==============================] - 18s 47ms/step - loss: 0.4045 - accuracy: 0.8383 - val_loss: 0.4374 - val_accuracy: 0.8277
Epoch 6/20
391/391 [==============================] - 18s 47ms/step - loss: 0.3852 - accuracy: 0.8449 - val_loss: 0.7906 - val_accuracy: 0.8010
Epoch 7/20
391/391 [==============================] - 18s 47ms/step - loss: 0.3723 - accuracy: 0.8527 - val_loss: 0.4747 - val_accuracy:

In [9]:
score = model.evaluate(X_test,Y_test)
acc = model.evaluate(X_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

1563/1563 [==============================] - 15s 10ms/step - loss: 0.2007 - accuracy: 0.9194
training accuracy:  0.919439971446991
testing accuracy:  0.8454177975654602
